In [1]:
import numpy as np
import pandas as pd

In [2]:
train_sero = pd.read_csv("./train_sero.csv")
company_start_year = pd.read_csv("./company_start_year.csv")

## 해당년도에 존재하는 회사만을 데이터에서 추출하여 rolling window

In [49]:
class Time_Series_Model:
    """
    원하는 기간을 입력하면 해당 기간에 맞춰 입력 데이터셋 생성
    """
    def __init__(self, term):
        self.term=term
    def is_exist(self, company_start_year):
        """
    해당 년도에 회사가 존재했는지에 관한 2000~2018년 까지 불리언 배열을 생성하는 함수
    company_start_year은 각 회사의 설립년도가 나와있는 데이터셋(2000년 이전에 설립된 회사는 2000년으로)"""
        
        num_company = company_start_year.shape[0]
        
        # 각 회사가 해당 년도에 있었는지를 반환하는 불리언 배열
        company_exist = [[] for i in range(num_company)] 
        for i in range(num_company):
            for year in range(2000,2019):

                # 만약 존재하면 1을 추가
                if company_start_year["start_year"][i]<=year: 
                    company_exist[i].append(True)

                # 만약 존재하지 않았을 시 0을 추가
                else:
                    company_exist[i].append(False)

        return np.array(company_exist)

    def make_dataset(self,data):
        company_exist = self.is_exist(company_start_year)
        cv = 18-self.term+1 # fold 개수

        for i in range(cv):
            # 해당 년도(시작년도 기준-2012~2014라면 2012 기준)에 존재하는 회사만을 추출
            one_bool_array = company_exist[:,i]
            bool_array = np.tile(one_bool_array,self.term)
            
            # 존재하는 회사 데이터 추출
            sub_data = np.array(data[i*(2191):2191*(i+self.term)][bool_array])
            num_data = one_bool_array.sum() # 해당 년도에 존재하는 회사 개수
            
            # 데이터 shape 변환
            sub_data = np.concatenate([sub_data[num_data*t:num_data*(t+1)] for t in range(term)], axis=1)
            
            X,y = sub_data[:,:-1], sub_data[:,-1]
            yield X,y
    

In [7]:
# target 값 분류 함수
def y_classification(x):
    if x==0:
        return x
    elif x==1:
        return x
    elif x>=2 and x<=4:
        return 2
    elif x>=5 and x<=7:
        return 3
    elif x>=8 and x<=10:
        return 4

### 어떤 기간이 가장 적절한지 비교

In [59]:
def value_sorted(x): # list 안의 list의 두번째 원소를 기준으로 정렬
    return x[1]

#### DecisionTree 사용

In [57]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
model = DecisionTreeClassifier()
terms = {i:0 for i in range(1,18)}

for term in terms.keys():
    n=0
    print(f"기간{term}")
    for idx, (X,y) in enumerate(Time_Series_Model(term=term).make_dataset(train_sero)):
        n+=1
        X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.3)
        model.fit(X_train, y_train)
        predicted = model.predict(X_val)
        accuracy = accuracy_score(y_val, predicted)
        terms[term] += accuracy
        print(f"{2000+idx}~{2000+idx+term-1} 까지의 정확도", accuracy)
    
    terms[term] /= n
    


기간1
2000~2000 까지의 정확도 0.3242630385487528
2001~2001 까지의 정확도 0.32264957264957267
2002~2002 까지의 정확도 0.26720647773279355
2003~2003 까지의 정확도 0.26232741617357
2004~2004 까지의 정확도 0.289272030651341
2005~2005 까지의 정확도 0.2725563909774436
2006~2006 까지의 정확도 0.31433823529411764
2007~2007 까지의 정확도 0.3039568345323741
2008~2008 까지의 정확도 0.30442477876106194
2009~2009 까지의 정확도 0.32578397212543553
2010~2010 까지의 정확도 0.3111111111111111
2011~2011 까지의 정확도 0.30084033613445377
2012~2012 까지의 정확도 0.29207920792079206
2013~2013 까지의 정확도 0.29200652528548127
2014~2014 까지의 정확도 0.3248
2015~2015 까지의 정확도 0.2962382445141066
2016~2016 까지의 정확도 0.2711864406779661
2017~2017 까지의 정확도 0.3009118541033435
기간2
2000~2001 까지의 정확도 0.35374149659863946
2001~2002 까지의 정확도 0.39316239316239315
2002~2003 까지의 정확도 0.3157894736842105
2003~2004 까지의 정확도 0.3037475345167653
2004~2005 까지의 정확도 0.32950191570881227
2005~2006 까지의 정확도 0.3815789473684211
2006~2007 까지의 정확도 0.35110294117647056
2007~2008 까지의 정확도 0.32014388489208634
2008~2009 까지의 정확도 0.359292035398

In [61]:
sorted(terms.items(), key=value_sorted, reverse=True)

[(8, 0.36812458966692835),
 (3, 0.36369283104846445),
 (11, 0.3632007768446445),
 (16, 0.36265954874977435),
 (17, 0.35949764521193095),
 (6, 0.3593768216921113),
 (5, 0.3584938138435669),
 (14, 0.3576709430322242),
 (2, 0.3572500261067045),
 (13, 0.35600083852262704),
 (9, 0.35596486829272056),
 (4, 0.3547976208976984),
 (15, 0.35390621524224763),
 (12, 0.35171108337628015),
 (7, 0.3484889085508473),
 (10, 0.3481737005098082),
 (1, 0.2986640259552065)]

#### RandomForest 사용

In [77]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
model = RandomForestClassifier(n_estimators=1000)
terms = {i:0 for i in range(1,18)}

for term in terms.keys():
    n=0
    print(f"기간{term}")
    
    for idx, (X,y) in enumerate(Time_Series_Model(term=term).make_dataset(train_sero)):
        n+=1

        X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.3)
        model.fit(X_train, y_train)
        predicted = model.predict(X_val)
        accuracy = accuracy_score(y_val, predicted)
        terms[term] += accuracy
        print(f"{2000+idx}~{2000+idx+term-1} 까지의 정확도", accuracy)
    
    terms[term] /= n
    


기간1
2000~2000 까지의 정확도 0.3333333333333333
2001~2001 까지의 정확도 0.37393162393162394
2002~2002 까지의 정확도 0.3825910931174089
2003~2003 까지의 정확도 0.34714003944773175
2004~2004 까지의 정확도 0.3620689655172414
2005~2005 까지의 정확도 0.4191729323308271
2006~2006 까지의 정확도 0.3602941176470588
2007~2007 까지의 정확도 0.3471223021582734
2008~2008 까지의 정확도 0.3734513274336283
2009~2009 까지의 정확도 0.35714285714285715
2010~2010 까지의 정확도 0.35555555555555557
2011~2011 까지의 정확도 0.37478991596638656
2012~2012 까지의 정확도 0.33663366336633666
2013~2013 까지의 정확도 0.3539967373572594
2014~2014 까지의 정확도 0.3728
2015~2015 까지의 정확도 0.38087774294670845
2016~2016 까지의 정확도 0.362095531587057
2017~2017 까지의 정확도 0.3738601823708207
기간2
2000~2001 까지의 정확도 0.4467120181405896
2001~2002 까지의 정확도 0.48504273504273504
2002~2003 까지의 정확도 0.4291497975708502
2003~2004 까지의 정확도 0.4161735700197239
2004~2005 까지의 정확도 0.4329501915708812
2005~2006 까지의 정확도 0.42293233082706766
2006~2007 까지의 정확도 0.40808823529411764
2007~2008 까지의 정확도 0.40827338129496404
2008~2009 까지의 정확도 0.469026548672

In [78]:
sorted(terms.items(), key=value_sorted, reverse=True)

[(2, 0.4431329383502976),
 (3, 0.43857899806102646),
 (4, 0.43693147536930266),
 (5, 0.42465222902551353),
 (11, 0.41972794057972557),
 (6, 0.4157222101467595),
 (17, 0.41487877202162915),
 (8, 0.4075675405735338),
 (9, 0.40744247632191327),
 (10, 0.40250386428449403),
 (12, 0.4000172320691126),
 (7, 0.3999561909627396),
 (15, 0.3980502379504693),
 (14, 0.39604490185053),
 (13, 0.38774633048032064),
 (16, 0.3785524556201248),
 (1, 0.3648254400672282)]

#### SVM

In [74]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
model = SVC(kernel="rbf", gamma=5, C=10)
terms = {i:0 for i in range(1,18)}

for term in terms.keys():
    n=0
    print(f"기간{term}")
    for idx, (X,y) in enumerate(Time_Series_Model(term=term).make_dataset(train_sero)):
        n+=1
        X = StandardScaler().fit_transform(X)
            
        X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.3)
        model.fit(X_train, y_train)
        predicted = model.predict(X_val)
        accuracy = accuracy_score(y_val, predicted)
        terms[term] += accuracy
        print(f"{2000+idx}~{2000+idx+term-1} 까지의 정확도", accuracy)
    
    terms[term] /= n
    


기간1
2000~2000 까지의 정확도 0.23582766439909297
2001~2001 까지의 정확도 0.22008547008547008
2002~2002 까지의 정확도 0.22874493927125505
2003~2003 까지의 정확도 0.23274161735700197
2004~2004 까지의 정확도 0.23563218390804597
2005~2005 까지의 정확도 0.2274436090225564
2006~2006 까지의 정확도 0.22794117647058823
2007~2007 까지의 정확도 0.22661870503597123
2008~2008 까지의 정확도 0.2336283185840708
2009~2009 까지의 정확도 0.2038327526132404
2010~2010 까지의 정확도 0.24444444444444444
2011~2011 까지의 정확도 0.21176470588235294
2012~2012 까지의 정확도 0.21122112211221122
2013~2013 까지의 정확도 0.21533442088091354
2014~2014 까지의 정확도 0.2048
2015~2015 까지의 정확도 0.21316614420062696
2016~2016 까지의 정확도 0.21263482280431434
2017~2017 까지의 정확도 0.21124620060790272
기간2
2000~2001 까지의 정확도 0.24489795918367346
2001~2002 까지의 정확도 0.23076923076923078
2002~2003 까지의 정확도 0.22469635627530365
2003~2004 까지의 정확도 0.29191321499013806
2004~2005 까지의 정확도 0.25862068965517243
2005~2006 까지의 정확도 0.2725563909774436
2006~2007 까지의 정확도 0.22058823529411764
2007~2008 까지의 정확도 0.22122302158273383
2008~2009 까지의 정확도 0.2

In [75]:
sorted(terms.items(), key=value_sorted, reverse=True)

[(2, 0.23909681546193995),
 (3, 0.22911457299820234),
 (5, 0.22611632171825002),
 (1, 0.22206157209333666),
 (9, 0.22180738658000107),
 (4, 0.21856615059315335),
 (7, 0.21677626418362425),
 (6, 0.2165644813188862),
 (8, 0.212625964468186),
 (15, 0.20910087277929962),
 (11, 0.20908855308312602),
 (17, 0.20907465550322696),
 (10, 0.20084975260481797),
 (16, 0.19600971292700617),
 (14, 0.19430863207513055),
 (12, 0.19342554897089428),
 (13, 0.18143609847255568)]

#### LogisticRegression 사용

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
model = LogisticRegression()
terms = {i:0 for i in range(1,18)}

for term in terms.keys():
    n=0
    print(f"기간{term}")
    for idx, (X,y) in enumerate(Time_Series_Model(term=term).make_dataset(train_sero)):
        n+=1
        
        X = StandardScaler().fit_transform(X)
        X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.3)
    
        model.fit(X_train, y_train)
        predicted = model.predict(X_val)
        accuracy = accuracy_score(y_val, predicted)
        terms[term] += accuracy
        print(f"{2000+idx}~{2000+idx+term-1} 까지의 정확도", accuracy)
    
    terms[term] /= n
    


기간1


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2000~2000 까지의 정확도 0.31746031746031744


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2001~2001 까지의 정확도 0.33974358974358976


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  ext

2001~2016 까지의 정확도 0.26495726495726496
2002~2017 까지의 정확도 0.012145748987854251
기간17


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2000~2016 까지의 정확도 0.2947845804988662
2001~2017 까지의 정확도 0.2777777777777778


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [65]:
sorted(terms.items(), key=value_sorted, reverse=True) 

[(1, 0.2909184944165901),
 (17, 0.28628117913832196),
 (2, 0.2848718233871878),
 (3, 0.2848082345514785),
 (13, 0.2498212793280702),
 (4, 0.23703574511756711),
 (15, 0.23283064133989526),
 (5, 0.23147491371667098),
 (8, 0.2084288410872216),
 (16, 0.20045535083129065),
 (6, 0.19764891180177693),
 (14, 0.188320615960671),
 (9, 0.1709834599900531),
 (11, 0.16515484599921038),
 (10, 0.15535129239946577),
 (7, 0.14969135825968402),
 (12, 0.11051130119247733)]

- 모델별 : RandomForest가 가장 높은 정확도 -> 44.3%
- 기간별 : 대부분의 모델에서 2개년 정도가 가장 높은 정확도